# Implementing Deep Dream with Keras

--- 
## Introduction
In this post, we are going to implement the Deep Dream algorithm in Keras.

> DeepDream is a computer vision program created by Google engineer Alexander Mordvintsev which uses a convolutional neural network to find and enhance patterns in images via algorithmic pareidolia, thus creating a dream-like hallucinogenic appearance in the deliberately over-processed images ([source](https://en.wikipedia.org/wiki/DeepDream))

Sound complicated? Whether you know it or not, you've probably seen deep dream before.

![Deep Dream Image 2](./images/deep_dream_2.jpg)

Deep dream uses the representations learned by convolutional neural networks to to modify the input image and introduce trippy, dream-like artifacts. The idea is simple, at each iteration of training, modify the input image to increase the output of a given layer. If that layer was trained to recognize dogs, then your image will slowly start to contain dog-like patterns.

---
## Choosing a Model

DeepDream can work with any trained convnet, but different convnets will have learned different features, so they will give different results. The original DeepDream implementation used an Inception model, so we will use InceptionV3 which is included with Keras. 

In [1]:
from keras.applications import InceptionV3
from keras import backend as K 

# Disable training 
K.set_learning_phase(0)

# Load the pretrained model, we don't need the output layers
model = InceptionV3(weights='imagenet', include_top=False)

Using TensorFlow backend.


---
## Defining a Loss Function

To update our image, we will use Keras to maximize the average output of a number of layers. The layers we choose to maximize will change the results of the algorithms. If we use earlier layers, which recognize edges and textures, then we will introduce simple patterns into our image. If we use later layers, which recognize birds and dogs, we will introduce more complex patterns into our image. We will combine the results of several different layers to make our model a bit more diverse. Note that these layer names can be discovered by using `model.summary()`.

In [141]:
layer_coefficients = {
    'mixed1': 1.0, 
    'mixed2': 1.0,
    'mixed3': 1.0,
}

Now that we've defined how much to weight each layer, we will define the loss function. The loss function is simply the sum of the L2 norms of the layers we selected above. The L2 norm of each layer is scaled by the coefficient we selected above. Note that we are keeping all of the channels in the output, but we are removing the borders from each channel.

In [142]:
# Dictionary: layer name => layer
layers = {layer.name: layer for layer in model.layers}

# Store the loss 
loss = K.variable(0.)

# Iterate through our chosen layers and add to loss function 
for layer_name, layer_coeff in layer_coefficients.items():
    activation = layers[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + layer_coeff * K.sum(K.square(activation[:, :, :, :])) / scaling 

---
## The Gradient Ascent Process
The gradient ascent process is where we actually modify the input image. Typically, when training a neural network, we use _gradient descent_. This minimizes a loss function with respect to the weights and biases in our network. For DeepDream, we are going to use _gradient descent_ to maximize our loss function with respect to the input images. This will cause the input image – the 'dream' – to form patterns that our chosen layers recognize.

In [143]:
# Hold the input image 
dream = model.input

# Compute the gradients with regard to the loss and normalize 
grads = K.gradients(loss, dream)[0]
grads = grads / K.maximum(K.mean(K.abs(grads)), 1e-7)

# Tells Keras we want the loss and gradient for a given image 
loss_and_grads = K.function([dream], [loss, grads])

def get_loss_and_gradients(image):
    output = loss_and_grads([image])
    loss = output[0]
    gradient = output[1]
    return loss, gradient

# Run Gradient Ascent 
def gradient_ascent(image, iterations, step, max_loss=None, verbose=True):
    for i in range(iterations):
        loss, gradient = get_loss_and_gradients(image)
        if max_loss is not None and loss > max_loss:
            break 
        if verbose:
            print('Loss at iteration', i, 'is', loss)
        image = image + step * gradient # the actual update
    return image 

---
## The DeepDream Algorithm
The actual DeepDream algorithm runs on the image at a number of differnt scales (called octaves). We apply gradient ascent to the image at the initial scale, scale it up by the determined amount (1.4 = 40%), then we reinsert lost detail (due to resizing) and repeat the process. For example, the default parameters shown below will run the image at 3 different sizes going from smallest to largest. After running gradient ascent, the image size will be increased by 40%, and the process will repeat.

In [144]:
import os
import numpy as np
from helpers.helpers import *

def dream(image_path, step=0.01, num_octaves=3, octave_scale=1.4, iterations=20, max_loss=10, verbose=True):
    """Implements the deep dream algorithm."""
    # Load the image
    img = preprocess_image(image_path)
    
    # Define what scales to run gradient descent
    image_shapes = image_octaves(img, octave_scale, num_octaves)
    
    # Create image copies
    original_img = np.copy(img)
    shrunk_original_img = resize_image(img, image_shapes[0])
    
    # Process the image at each scale
    for shape in image_shapes:
        print('Processing image shape', shape)
        img = resize_image(img, shape)
        img = gradient_ascent(img, iterations=iterations, step=step, 
                              max_loss=max_loss, verbose=verbose)
        
        # Insert 'detail' into resized image
        upscaled_shrunk_original_img = resize_image(shrunk_original_img, shape)
        same_size_original = resize_image(original_img, shape)
        lost_detail = same_size_original - upscaled_shrunk_original_img
        img += lost_detail 
        shrunk_original_img = resize_image(original_img, shape)
        save_image(img, filename='dream_at_scale_' + str(shape) + '.png')
    
    ext = os.path.splitext(os.path.basename(image_path))[0]
    filename = 'final_dream_' + ext + '.png'
    save_image(img, filename=filename)

Now, we can use this function to generate dreams. Feel free to modify the parameters or supply your own images.

In [148]:
dream('./inputs/shiba.jpg', step=0.01, num_octaves=3, iterations=25, octave_scale=1.2, max_loss=None, verbose=False)

Processing image shape (416, 416)
Processing image shape (500, 500)
Processing image shape (600, 600)


---
## Some Examples

### Original Image
![Original Shiba Image](./inputs/shiba.jpg)

### Example 1
```
layer_coefficients = {
    'conv2d_1': 1.0, 
}
dream('./inputs/shiba.jpg', step=0.01, num_octaves=1, iterations=20, max_loss=None, verbose=True)
```
![Example 1](./outputs/final_dream_shiba_conv2d_1.png)


### Example 2
```
layer_coefficients = {
    'conv2d_4': 1.0, 
}
dream('./inputs/shiba.jpg', step=0.01, num_octaves=1, iterations=20, max_loss=None, verbose=True)
```
![Example 2](./outputs/final_dream_shiba_conv2d_4.png)


### Example 3
```
layer_coefficients = {
    'mixed1': 1.0, 
}
dream('./inputs/shiba.jpg', step=0.01, num_octaves=1, iterations=20, max_loss=None, verbose=True)
```
![Example 3](./outputs/final_dream_shiba_mixed1.png)


### Example 4
```
layer_coefficients = {
    'mixed10': 1.0, 
}
dream('./inputs/shiba.jpg', step=0.01, num_octaves=1, iterations=20, max_loss=None, verbose=True)
```
![Example 4](./outputs/final_dream_shiba_mixed10.png)


### Example 5
```
layer_coefficients = {
    'conv2d_1': 1.0, 
    'conv2d_4': 1.0,
    'conv2d_12': 1.0,
}
dream('./inputs/shiba.jpg', step=0.01, num_octaves=3, iterations=20, max_loss=100, verbose=True)
```
![Example 5](./outputs/final_dream_shiba_3_layers_3_octaves.png)


### Example 6
```
layer_coefficients = {
    'mixed1': 1.0, 
    'mixed2': 1.0,
    'mixed3': 1.0,
}
dream('./inputs/shiba.jpg', step=0.01, num_octaves=1, iterations=50, max_loss=None, verbose=False)
```
![Example 6](./outputs/final_dream_shiba.png)

--- 
## References
The photo of the Shiba can be found [here](https://t2.ea.ltmcdn.com/en/images/3/5/1/img_shedding_of_shiba_inus_153_600.jpg)